In [1]:
import tensorflow as tf
import timeit
from datetime import datetime

You create and run a graph in TensorFlow by using tf.function, either as a direct call or as a decorator. tf.function 
takes a regular function as input and returns a Function. A Function is a Python callable that builds TensorFlow graphs from the Python function. 
You use a Function in the same way as its Python equivalent.

# Define a Python function

In [2]:
def regular_function(x, y, b):
    x = tf.matmul(x, y)
    x += b
    return x

Make some tensors

In [3]:
x1 = tf.constant([[1., 2.]])
y1 = tf.constant([[2.], [3.]])
b1 = tf.constant(4.)

In [6]:
regular_value = regular_function(x1, y1, b1).numpy()
regular_value

array([[12.]], dtype=float32)

`a_function_that_uses_a_graph` is a TensorFlow `Function`

In [7]:
function_that_usses_tensorflow = tf.function(regular_function)

In [8]:
tensorflow_function_value = function_that_usses_tensorflow(x1, y1, b1).numpy()
tensorflow_function_value

array([[12.]], dtype=float32)

In [9]:
assert(regular_value == tensorflow_function_value)

# Use of decorator

In [10]:
def inner_function(x, y, b):
    x = tf.matmul(x, y)
    x += b
    return x

Use the decorator to make `outer_function` a `Function`

In [11]:
@tf.function
def outer_function(x):
    y = tf.constant([ [2.], [3.] ])
    b = tf.constant(4.)
    return inner_function(x, y, b)

Note that the callable will create a graph that includes `inner_function` as well as `outer_function`.

In [12]:
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

# Converting Python functions to graphs
Any function you write with TensorFlow will contain a mixture of built-in TF operations and Python logic, such as if-then clauses, 
loops, break, return, continue, and more. While TensorFlow operations are easily captured by a tf.Graph, Python-specific logic needs to undergo an extra step in order to become part of the graph. tf.function uses a library called AutoGraph (tf.autograph) to convert Python code into graph-generating code.

In [13]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0

`tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.

In [14]:
tf_simple_relu = tf.function(simple_relu)
print('First branch, with graph', tf_simple_relu(tf.constant(1)).numpy())
print('First branch, with graph', tf_simple_relu(tf.constant(-1)).numpy())

First branch, with graph 1
First branch, with graph 0


Though it is unlikely that you will need to view graphs directly, you can inspect the outputs to check the exact results. These are not easy to read, so no need to look too carefully!
This is the graph-generating output of AutoGraph

In [16]:
print(tf.autograph.to_code(simple_relu))

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal retval_, do_return
            (do_return, retval_) = vars_

        def if_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

This is the graph itself.

In [17]:
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "Tcond"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "Tin"
    value {
      list {
        type: DT_INT32
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_BOOL
        type: DT_INT32
      }
    }
  

A tf.Graph is specialized to a specific type of inputs (for example, tensors with a specific dtype or objects with the same id()).

Each time you invoke a Function with a set of arguments that can't be handled by any of its existing graphs (such as arguments with new dtypes or incompatible shapes), Function creates a new tf.Graph specialized to those new arguments. The type specification of a tf.Graph's inputs is known as its input signature or just a signature. For more information regarding when a new tf.Graph is generated and how that can be controlled, see the rules of retracing.

In [18]:
@tf.function
def my_relu(x):
  return tf.maximum(0., x)

In [19]:
print(my_relu(tf.constant(5.5)))
print(my_relu([1, -1]))
print(my_relu(tf.constant([3., -3.])))

tf.Tensor(5.5, shape=(), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor([3. 0.], shape=(2,), dtype=float32)


# Graph execution vs. eager execution

Difference of eager and graph execution
The code in a Function can be executed both eagerly and as a graph. By default, Function executes its code as a graph.
However, Function can behave differently under graph and eager execution. The Python print function is one example of how these two modes differ. 
Let's check out what happens when you insert a print statement to your function and call it repeatedly.

In [20]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)

In [21]:
@tf.function
def get_MSE(y_true, y_pred):
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)

In [23]:
get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=41>

To verify that your Function's graph is doing the same computation as its equivalent Python function, you can make it execute eagerly with

In [24]:
tf.config.run_functions_eagerly(True)

In [25]:
get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=41>

In [26]:
tf.config.run_functions_eagerly(False)

However, Function can behave differently under graph and eager execution. The Python print function is one example of how these two modes differ. Let's check out what happens when you insert a print statement to your function and call it repeatedly.

In [28]:
def get_MSE(y_true, y_pred):
    print("Calculating MSE!")
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)

In [29]:
print('Graph execution')
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

Graph execution
Calculating MSE!
Calculating MSE!
Calculating MSE!


In [30]:
print('Eager execution')
tf.config.run_functions_eagerly(True)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
tf.config.run_functions_eagerly(False)

Eager execution
Calculating MSE!
Calculating MSE!
Calculating MSE!
